In [8]:
from langchain_community.llms import Ollama
llm2 = Ollama(model="llama3.1:latest")
# llm2 = Ollama(model="nous-hermes2:10.7b")

# llm2 = Ollama(model="gemma:2b")

#

In [3]:
llm2.invoke("hello")

'Hello! How can I assist you today?'

In [47]:
llm2.invoke("""You are a helpful assistant that converts cause-effect relationships into a tree-like structure.

Input: 
Cause A: Effect X
Cause B: Effect Y
Effect X: Effect Z
Effect Y: Effect W
Effect W: Effect V

Output:
""")

'Here is the converted cause-effect relationship in a tree-like structure:\n\n\n            Cause A\n           /    \\\n       Effect X   Cause B\n          |         |\n      Effect Z     Effect Y\n             |        |\n        Effect V  Effect W\n               |\n\nThis represents the causal chain from Cause A to Effect Z, and from Cause B to Effect V.'

In [12]:
from langchain.agents import tool


@tool
def get_word_length(word: str) -> int:
    """Returns the length o."""
    return len(word)


get_word_length.invoke("abcu")

4

In [4]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant, but don't know current events",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [4]:
from langchain_openai import OpenAI

In [9]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [ ]:

"You are a helpful assistant that converts cause-effect relationships into a tree-like structure(keep the type of every input) Input"

In [ ]:
"""cause: Always -> Effect :Get the cbit test result
Effect: Get the cbit test result -> cause: CBIT Test Result is equal to E_CBIT_Fatal_Error
cause: CBIT Test Result is equal to E_CBIT_Fatal_Error -> Effect: Set (V_Modes_SW_Mode) to E_Modes_Fail
Effect: Get the cbit test result -> cause: all other cases
cause all other cases: -> Effect: get the ovsp config table integrity status
Effect: get the ovsp config table integrity status -> cause: OVSP Config table Integrity Status is equal to E_Config_Integrity_KO
cause: OVSP Config table Integrity Status is equal to E_Config_Integrity_KO -> Effect: Set (V_Modes_SW_Mode) to E_Modes_Degraded
Effect: get the ovsp config table integrity status -> cause: all other cases
cause: all other cases -> Effect: get the can triggered test 
"""

In [9]:
prompt = PromptTemplate(
    input_variables=["input"],
    template=""" You are a helpful assistant that converts cause-effect relationships into a clear, well-structured tree format. For each relationship, clearly label whether it's a "Cause" or an "Effect" at every level. 

Follow these rules:
- Each cause is listed only once, and all related effects are nested under that cause.
- The cause should be labeled "(Cause)" and each effect labeled "(Effect)".
- Make sure to preserve the hierarchy of relationships.
{input}
Output::"""
)

chain = LLMChain(llm=llm, prompt=prompt)

# Ask a question
input = """ 
cause: V Modes SW Mode is equal to E Modes Init -> Effect : CALL Modes Manage Init Tr
Effect: CALL Modes Manage Init Tr -> cause: V Modes SW Mode is equal to E Modes Fail
cause: V Modes SW Mode is equal to E Modes Operational -> Effect: CALL Modes Manage Operational Tr
Effect: CALL Modes Manage Operational Tr -> cause: V Modes SW Mode is equal to E Modes Fail
cause  IV Modes SW Mode is equal to E Modes Degraded: -> Effect: CALL Modes Manage Degraded Tr
Effect: CALL Modes Manage Degraded Tr -> cause: V Modes SW Mode is equal to E Modes Fail
cause: V Modes SW Mode is equal to E Modes TriggeredTest -> Effect: CALL Modes Manage TriggeredTest Tr
Effect: CALL Modes Manage TriggeredTest Tr -> cause: V Modes SW Mode is equal to E Modes Fail
cause : all other cases -> Effect: set V Modes SW Modes to E Modes Fail
cause: V Modes SW Mode is equal to E Modes Fail -> Effect: CALL DryGPIO Write with IN DRVGPIO_OUT_OSARM_CMD and E OVSP SW Cmd On
Effect: set V Modes SW Modes to E Modes Fail -> cause: all other cases
cause: all other cases -> Effect: No effect
Effect: CALL DryGPIO Write with IN DRVGPIO_OUT_OSARM_CMD and E OVSP SW Cmd On -> cause:Logbook failure to write is FALSE
cause:Logbook failure to write is FALSE -> Effect: Wait until an external interrupt is executed
Effect: CALL DryGPIO Write with IN DRVGPIO_OUT_OSARM_CMD and E OVSP SW Cmd On ->cause:all other cases
cause:all other cases -> Effect: No effect
"""
answer1 = chain.run(input)
print(answer1)


Cause: V Modes SW Mode is equal to E Modes Init
Effect: CALL Modes Manage Init Tr
Cause: V Modes SW Mode is equal to E Modes Fail
Cause: V Modes SW Mode is equal to E Modes Operational
Effect: CALL Modes Manage Operational Tr
Cause: IV Modes SW Mode is equal to E Modes Degraded
Effect: CALL Modes Manage Degraded Tr
Cause: V Modes SW Mode is equal to E Modes TriggeredTest
Effect: CALL Modes Manage TriggeredTest Tr
Effect: set V Modes SW Modes to E Modes Fail
Cause: all other cases
Effect: CALL DryGPIO Write with IN DRVGPIO_OUT_OSARM_CMD and E OVSP SW Cmd On
Cause: Logbook failure to write is FALSE
Effect: Wait until an external interrupt is executed
Cause: all other cases
Effect: No effect


In [ ]:
template="""You are a helpful assistant that converts cause-effect relationships into a clear, well-structured tree format. Each cause should only appear once, with its effects nested under it. Label each node clearly as "Cause" or "Effect" and preserve the hierarchy of relationships.

For example:

Cause: V Modes SW Mode is equal to E Modes Init
    Effect: CALL Modes Manage Init Tr
    Cause: V Modes SW Mode is equal to E Modes Fail
        Effect: <Subsequent Effect Here>
Cause: V Modes SW Mode is equal to E Modes Operational
    Effect: CALL Modes Manage Operational Tr

{input}

Output::
"""

In [ ]:
prompt = PromptTemplate(
    input_variables=["input"],
   template="""You are a helpful assistant that converts cause-effect relationships into a clear, well-structured tree format.with its effects nested under it. Label each node clearly as "Cause" or "Effect" and preserve the hierarchy of relationships.

For example:
the input is 
cause: A->Effect: 1
Effect: 1->cause: B
cause: B->Effect: 2
Effect:2->cause C
cause C->Effect: 3
cause:D->Effect:4
Effect:4->cause : E
cause : E->Effect:5 
And the output is 
Cause: A
    Effect: 1
    Cause: B
        Effect: 2
        Cause: C
            Effect: 3
Cause: D
    Effect: 4
    Cause: E
        Effect: 5

{input}

Output::
"""
)

chain = LLMChain(llm=llm2, prompt=prompt)

# Ask a question
input = """ 
first ligne of cause 
cause: V Modes SW Mode is equal to E Modes Init -> Effect : CALL Modes Manage Init Tr
cause: V Modes SW Mode is equal to E Modes Operational -> Effect: CALL Modes Manage Operational Tr
cause  IV Modes SW Mode is equal to E Modes Degraded: -> Effect: CALL Modes Manage Degraded Tr
cause: V Modes SW Mode is equal to E Modes TriggeredTest -> Effect: CALL Modes Manage TriggeredTest Tr
cause : all other cases -> Effect: set V Modes SW Modes to E Modes Fail
seconde ligne of cause
Effect: CALL Modes Manage Init Tr -> cause: V Modes SW Mode is equal to E Modes Fail
Effect: CALL Modes Manage Operational Tr -> cause: V Modes SW Mode is equal to E Modes Fail
Effect: CALL Modes Manage Degraded Tr -> cause: V Modes SW Mode is equal to E Modes Fail
Effect: CALL Modes Manage TriggeredTest Tr -> cause: V Modes SW Mode is equal to E Modes Fail
cause: V Modes SW Mode is equal to E Modes Fail -> Effect: CALL DryGPIO Write with IN DRVGPIO_OUT_OSARM_CMD and E OVSP SW Cmd On
cause: all other cases -> Effect: No effect
third ligne of cause
Effect: CALL DryGPIO Write with IN DRVGPIO_OUT_OSARM_CMD and E OVSP SW Cmd On -> cause:Logbook failure to write is FALSE
cause:Logbook failure to write is FALSE -> Effect: Wait until an external interrupt is executed
cause:all other cases -> Effect: No effect

"""
answer1 = chain.run(input)
print(answer1)

In [6]:
prompt = PromptTemplate(
    input_variables=["input"],
    template="""You are a smart assistant that converts cause-effect trees into multiple test scenarios. For each cause-effect path:

1.Assign a unique name in the format "UFT<number>".
2.For each cause in the path, label it:
  .T (True) if the condition is satisfied,
  .F (False) if the condition is not satisfied,
  .# if the cause condition doesn't matter.
3.For each effect in the path, label it:
  .X if the effect occurs,
  .- if the effect does not occur.
The scenarios should follow this format:
 .Scenario name (UFT1, UFT2, etc.)
 .For each cause, indicate whether it is True (T), False (F), or doesn't matter (#).
 .For each effect, indicate if it occurs (X) or doesn't occur (-).  

{input}
Output:
"""
)

chain = LLMChain(llm=llm2, prompt=prompt)

# Ask a question
input = """ 
cause 1.1-> effect 1
cause 1.2-> effect 2
cause 1.3-> effect 3
cause 1.4-> effect 4
cause 1 all other cases -> effect 5
effect 1.1 ->cause 2.1
effect 1.2 ->cause 2.1
effect 1.3 ->cause 2.1
effect 1.4 ->cause 2.1
cause 2 all other cases -> no effect
cause 2.1 -> effect 6
effect 6 -> cause 3.1
cause 3.1-> effect 6
cause 3 all other cases -> no effect



"""
answer1 = chain.run(input)
print(answer1)

C:\Users\malek\AppData\Local\Temp\ipykernel_23704\3692924458.py:22: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  chain = LLMChain(llm=llm2, prompt=prompt)
C:\Users\malek\AppData\Local\Temp\ipykernel_23704\3692924458.py:44: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  answer1 = chain.run(input)


Here are the test scenarios for each cause-effect path:

**UFT1**

* Cause 1.1: T
* Effect 1: X

**UFT2**

* Cause 1.2: T
* Effect 1: -
* Effect 2: X

**UFT3**

* Cause 1.3: T
* Effect 2: -
* Effect 3: X

**UFT4**

* Cause 1.4: T
* Effect 3: -
* Effect 4: X

**UFT5**

* Cause 1 (all other cases): # 
* Effect 4: X
* Effect 5: X

**UFT6**

* Cause 2 (all other cases): #
* Cause 2.1: T
* Effect 6: X

**UFT7**

* Cause 3 (all other cases): # 
* Cause 3.1: T
* Effect 6: -


In [ ]:
prompt = PromptTemplate(
    input_variables=["input"],
    template="""You are a smart assistant that converts cause-effect trees into multiple test scenarios. For each cause-effect path:

1.Assign a unique name in the format "UFT<number>".
2.For each cause in the path, label it:
  .T (True) if the condition is satisfied,
  .F (False) if the condition is not satisfied,
  .# if the cause condition doesn't matter.
3.For each effect in the path, label it:
  .X if the effect occurs,
  .- if the effect does not occur.
The scenarios should follow this format:
 .Scenario name (UFT1, UFT2, etc.)
 .For each cause, indicate whether it is True (T), False (F), or doesn't matter (#).
 .For each effect, indicate if it occurs (X) or doesn't occur (-).  

{input}
Output:
"""
)

chain = LLMChain(llm=llm2, prompt=prompt)

# Ask a question
input = """ 
cause 1.1-> effect 1
cause 1.2-> effect 2
cause 1.3-> effect 3
cause 1.4-> effect 4
cause 1 all other cases -> effect 5
effect 1.1 ->cause 2.1
effect 1.2 ->cause 2.1
effect 1.3 ->cause 2.1
effect 1.4 ->cause 2.1
cause 2 all other cases -> no effect
cause 2.1 -> effect 6
effect 6 -> cause 3.1
cause 3.1-> effect 6
cause 3 all other cases -> no effect



"""
answer1 = chain.run(input)
print(answer1)

lkhedma

In [10]:
prompt = PromptTemplate(
    input_variables=["input"],
    template="""You are a smart assistant that converts cause-effect trees into multiple test scenarios. For each cause-effect path:

1.Assign a unique name in the format "UFT<number>".
2.For each cause in the path, label it:
  .T (True) if the condition is satisfied,
  .F (False) if the condition is not satisfied,
  .# if the cause condition doesn't matter.
3.For each effect in the path, label it:
  .X if the effect occurs,
  .- if the effect does not occur.
The scenarios should follow this format:
 .Scenario name (UFT1, UFT2, etc.)
 .For each cause, indicate whether it is True (T), False (F), or doesn't matter (#).
 .For each effect, indicate if it occurs (X) or doesn't occur (-).  

{input}
Output:
"""
)

chain = LLMChain(llm=llm2, prompt=prompt)

# Ask a question
input = """ 
cause 1.1-> effect 1
cause 1.2-> effect 2
cause 1.3-> effect 3
cause 1.4-> effect 4
cause 1 all other cases -> effect 5
effect 1.1 ->cause 2.1
effect 1.2 ->cause 2.1
effect 1.3 ->cause 2.1
effect 1.4 ->cause 2.1
cause 2 all other cases -> no effect
cause 2.1 -> effect 6
effect 6 -> cause 3.1
cause 3.1-> effect 6
cause 3 all other cases -> no effect



"""
answer1 = chain.run(input)
print(answer1)

Based on the provided cause-effect tree, I will generate multiple test scenarios as per your requirements.

Here is the output:


**Scenario UFT1**
Cause: 1.1 # - (True for cause 1.1, doesn't matter for others) 
Effect: 1 X (Effect 1 occurs)

**Scenario UFT2**
Cause: 1.2 T - (Effect 1.2 is True) 
Effect: 1 - (Effect 1 does not occur)

**Scenario UFT3**
Cause: 1.3 # X (Effect 3 occurs, doesn't matter for others) 
Effect: 3 - (Effect 3 does not occur)

**Scenario UFT4**
Cause: 1.4 F - (Effect 1.4 is False) 
Effect: 1 - (Effect 1 does not occur)

**Scenario UFT5**
Cause: 1 all other cases T X (All causes except 1 are True, effect 5 occurs)
Effect: 2 - (No effect from cause 2) 
Effect: 3 - (No effect from cause 3) 
Effect: 5 X (Effect 5 occurs)

**Scenario UFT6**
Cause: 1 all other cases # X (All causes except 1 are False, doesn't matter) 
Effect: 2 - (No effect from cause 2)
Effect: 3 - (No effect from cause 3)
Effect: 5 - (Effect 5 does not occur)

**Scenario UFT7**
Cause: 2.1 T X (Cause

In [11]:
prompt = PromptTemplate(
    input_variables=["input"],
    template="""You are a smart assistant that converts cause-effect trees into multiple test scenarios. For each cause-effect path:

1.Assign a unique name in the format "UFT<number>".
2.For each cause in the path, label it:
  .T (True) if the condition is satisfied,
  .F (False) if the condition is not satisfied,
  .# if the cause condition doesn't matter.
3.For each effect in the path, label it:
  .X if the effect occurs,
  .- if the effect does not occur.
The scenarios should follow this format:
 .Scenario name (UFT1, UFT2, etc.)
 .For each cause, indicate whether it is True (T), False (F), or doesn't matter (#).
 .For each effect, indicate if it occurs (X) or doesn't occur (-).  

{input}
Output:
"""
)

chain = LLMChain(llm=llm2, prompt=prompt)

# Ask a question
input = """ 
cause 1 always -> effect 1
effect 1 -> cause 2.1
cause 2.1 -> effect 2
cause 2 all other cases -> effect 3
effect 3 -> cause 3.1
cause 3.1 -> effect 4
cause 3 all other cases -> effect 5
effect 5 ->cause 4.1
cause 4.1 ->effect 6
cause 4 all other cases -> No effect
"""
answer1 = chain.run(input)
print(answer1)

Based on the provided cause-effect tree, I will generate multiple test scenarios as per your request.

Here are the scenarios:

**UFT1**
Cause: T (True) -> Effect: X (Effect occurs)
 Cause condition: cause 1 always satisfied
 Effect: effect 1 occurs

**UFT2**
Cause: # (No matter) -> Effect: - (Effect does not occur)
 Cause condition: Effect 1 doesn't matter, but still leads to the next cause
 Effect: effect 2 doesn't occur

**UFT3**
Cause: T (True) -> Effect: X (Effect occurs)
 Cause condition: Cause 2.1 is always satisfied
 Effect: effect 2 occurs

**UFT4**
Cause: # (No matter) -> Effect: - (Effect does not occur)
 Cause condition: Cause 2 all other cases don't matter, but still lead to the next effect
 Effect: effect 3 doesn't occur

**UFT5**
Cause: T (True) -> Effect: X (Effect occurs)
 Cause condition: Cause 3.1 is always satisfied
 Effect: effect 3 occurs

**UFT6**
Cause: # (No matter) -> Effect: - (Effect does not occur)
 Cause condition: Cause 3 all other cases don't matter, but